In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import matplotlib.pyplot as plt

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Rikitea,PF,2019-12-29 16:29:21,-23.12,-134.97,75.65,80,100,11.48,overcast clouds,0.0,0.0
1,1,Conakry,GN,2019-12-29 16:29:21,9.52,-13.70,87.80,62,0,11.41,haze,0.0,0.0
2,2,Mindelo,PT,2019-12-29 16:29:22,41.31,-8.72,63.00,55,0,0.96,clear sky,0.0,0.0
3,3,Comodoro Rivadavia,AR,2019-12-29 16:27:03,-45.87,-67.48,71.60,43,14,9.17,few clouds,0.0,0.0
4,4,Navahrudak,BY,2019-12-29 16:29:22,53.59,25.82,25.03,82,19,3.85,few clouds,0.0,0.0


In [3]:
#df = city_data_df
user_input_min_int = float(input("What is the Minimum Temperature?"))
user_input_max_int = float(input("What is the Maximum Temperature?"))
user_input_snow = str(input("Do you want it to be snowing? (yes/no)\n"))
user_input_rain = str(input("Do you want it to be raining? (yes/no)\n"))


What is the Minimum Temperature?0
What is the Maximum Temperature?90
Do you want it to be snowing? (yes/no)
no
Do you want it to be raining? (yes/no)
no


In [4]:
#Filter based on snow
if ( user_input_snow.upper()) ==  'YES':
    city_data_df_1 = (city_data_df["Snow inches (last 3 hrs)"] > 0)
    city_data_df_3 = city_data_df[city_data_df_1]
else:
    city_data_df_3 =  city_data_df

In [5]:
#Filter based on rain
if ( user_input_rain.upper()) ==  'YES':
    city_data_df_4 = (city_data_df_3["Rain inches (last 3 hrs)"] > 0)
    city_data_df_5 = city_data_df_3[city_data_df_4]
else:
    city_data_df_5 =  city_data_df_3  

In [6]:
#Filter based on temperature
city_data_df_2 =  (city_data_df_5["Max Temp"] > user_input_min_int) & (city_data_df_5["Max Temp"] < user_input_max_int)
city_data_df_6 = city_data_df_5[city_data_df_2]
city_data_df_6.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Rikitea,PF,2019-12-29 16:29:21,-23.12,-134.97,75.65,80,100,11.48,overcast clouds,0.0,0.0
1,1,Conakry,GN,2019-12-29 16:29:21,9.52,-13.70,87.80,62,0,11.41,haze,0.0,0.0
2,2,Mindelo,PT,2019-12-29 16:29:22,41.31,-8.72,63.00,55,0,0.96,clear sky,0.0,0.0
3,3,Comodoro Rivadavia,AR,2019-12-29 16:27:03,-45.87,-67.48,71.60,43,14,9.17,few clouds,0.0,0.0
4,4,Navahrudak,BY,2019-12-29 16:29:22,53.59,25.82,25.03,82,19,3.85,few clouds,0.0,0.0


In [7]:
#Dataframe for filtered data
vacation_df = city_data_df_6[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()
vacation_df["Hotel Name"] = ""
vacation_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.65,overcast clouds,-23.12,-134.97,
1,Conakry,GN,87.80,haze,9.52,-13.70,
2,Mindelo,PT,63.00,clear sky,41.31,-8.72,
3,Comodoro Rivadavia,AR,71.60,few clouds,-45.87,-67.48,
4,Navahrudak,BY,25.03,few clouds,53.59,25.82,
...,...,...,...,...,...,...,...
548,Biloela,AU,64.65,overcast clouds,-24.40,150.51,
549,Malangali,TZ,61.30,overcast clouds,-8.57,34.85,
550,Shangqiu,CN,46.99,clear sky,34.42,115.65,
551,Tablac,PH,69.96,clear sky,17.22,120.48,


In [8]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [9]:
# Iterate through the DataFrame.
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

import urllib3
urllib3.disable_warnings()
for index, row in vacation_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params,verify=False).json()
    # Grab the first hotel from the results and store the name.
    try:
        vacation_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
vacation_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.65,overcast clouds,-23.12,-134.97,Pension Maro'i
1,Conakry,GN,87.80,haze,9.52,-13.70,Golden Plazza
2,Mindelo,PT,63.00,clear sky,41.31,-8.72,Santana Hotel & Spa
3,Comodoro Rivadavia,AR,71.60,few clouds,-45.87,-67.48,Austral Hotel
4,Navahrudak,BY,25.03,few clouds,53.59,25.82,Guest House in Novogrudok
...,...,...,...,...,...,...,...
548,Biloela,AU,64.65,overcast clouds,-24.40,150.51,Silo Motor Inn
549,Malangali,TZ,61.30,overcast clouds,-8.57,34.85,
550,Shangqiu,CN,46.99,clear sky,34.42,115.65,7Days Inn Shangqiu Railway Station
551,Tablac,PH,69.96,clear sky,17.22,120.48,Ban-aw Resort


In [11]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} </dd>
</dl>
"""

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
vacation_df.to_csv(output_data_file, index_label="City_ID")